In [1]:
import numpy as np
import pandas as pd

import os
from scipy.integrate import trapezoid
import pathlib

In [2]:
base_dir = os.path.join(pathlib.Path.home(), 'ecen5407', 'ecen5407_project2')
pysam_pt = os.path.join(base_dir, "data", "PySam_Outputs", "baseline_system_output.csv")
pysam = pd.read_csv(pysam_pt)

In [3]:
load = pd.Series(pysam['Load (MW)'])

In [4]:
transmission_cost_per_mile = 6000000

In [5]:
wind_ac_rating = 61 # MWac
wind_op_costs = 30.85*wind_ac_rating*1000
wind_capex = 1536.87*wind_ac_rating*1000

geothermal_ac_rating = 87 # MW, only 77 MW will actually be available to grid
geothermal_cost_per_kwac = 6153.66
geothermal_capex = 26.6*transmission_cost_per_mile + geothermal_cost_per_kwac*geothermal_ac_rating*1000 # extra term reflects size-independent transmission costs
geothermal_op_costs = 118.41*geothermal_ac_rating*1000

In [6]:
# Scaling factors
pv_dc_power_rating = 230 # MWdc
battery_dc_power_rating = 30 # MW
battery_dc_capacity = 90 #Mwhdc
ilr = 1.33

inverter_dc_rating = 2579 # kWdc
inverter_efficiency = 0.97
inverter_ac_rating = inverter_dc_rating*inverter_efficiency # kWac
num_inv = round(pv_dc_power_rating*1000/(ilr*inverter_ac_rating))

area = pv_dc_power_rating*1000/0.206 # m2

# Component costs
inverter_cost = num_inv*inverter_ac_rating*38.72
module_cost = 295.68*pv_dc_power_rating*1000
battery_cost = 228*battery_dc_capacity

bos_cost_per_m2 = 27.07
bos_cost_per_kwdc = 156.16
bos_cost = bos_cost_per_m2*area + bos_cost_per_kwdc*pv_dc_power_rating*1000 # includes battery costs
installation_costs = 92.11*area # includes battery installation
overhead = 66.63*pv_dc_power_rating*1000 # Management + Contingency
permitting = 200000
engineering = 50000 + 3*area
interconnection = 85000 + 35*pv_dc_power_rating/ilr
land_purchase = 2.5*area
transmission = 25*transmission_cost_per_mile

# Total costs
hybrid_capex = module_cost + inverter_cost + battery_cost + bos_cost + installation_costs + overhead + permitting + engineering + interconnection + transmission + land_purchase
hybrid_op_costs = 48*pv_dc_power_rating*1000 + 180000

print(f"PV+Battery system CAPEX is ${round(hybrid_capex)}")

PV+Battery system CAPEX is $415499053


In [7]:
annual_op_costs =  wind_op_costs + geothermal_op_costs + hybrid_op_costs
print(f"Annual operating costs in 2026 are ${round(annual_op_costs)}")

Annual operating costs in 2026 are $23403520


In [8]:
inflation_rate = 1.0025
annual_op_costs_by_year = [annual_op_costs*inflation_rate**i for i in range(1, 26)]
total_op_costs = sum(annual_op_costs_by_year)
avg_annual_op_costs = total_op_costs/25
print(f"Total operating costs over plant lifetime are ${round(total_op_costs)}")
print(f"Average annual operating costs are ${round(avg_annual_op_costs)}")

Total operating costs over plant lifetime are $604489195
Average annual operating costs are $24179568


In [14]:
upfront_cost = geothermal_capex + wind_capex + hybrid_capex
real_interest_rate = 1 + (0.006 - 0.0025)
real_upfront_cost = upfront_cost*real_interest_rate**25 
avg_annual_upfront_payment = real_upfront_cost/25
print(f"Total CAPEX with interest and inflation are ${round(real_upfront_cost)}")
print(f"Average annual CAPEX payments are ${round(avg_annual_upfront_payment)}")

Total CAPEX with interest and inflation are $1314132057
Average annual CAPEX payments are $52565282


In [15]:
avg_annual_total_payment = avg_annual_upfront_payment + avg_annual_op_costs
print(f"Annual revenue required to meet CAPEX and operating costs is ${round(avg_annual_total_payment)}")

Annual revenue required to meet CAPEX and operating costs is $76744850


In [16]:
x = [i for i in range(0, 525600, 5)]
annual_load = trapezoid(load.values, x) # Mwh
avg_energy_rate = avg_annual_total_payment/(1000*annual_load)
print(f"Total annual load is {round(annual_load)} MWh")
print(f"Average energy rate is ${round(avg_energy_rate, 4)}/kWh")

Total annual load is 24463208 MWh
Average energy rate is $0.0031/kWh
